In [1]:
from metaflow import Flow, get_metadata
import re
import string

print("Current metadata provider: %s" % get_metadata())

Current metadata provider: local@/Users/sonoffungdark/Projects/merchantizer/merchantizer


### What happened in load.py?
The code tests to see if the data has already been downloaded locally. If not, it grabs it from the link offered in the problem statement. It then prints the shape (rows, columns) of the data. I next read in the loaded data with the cell below.

In [2]:
run = Flow('TransactionLoadFlow').latest_successful_run
print("Using run: %s" % str(run))

raw_df = run.data.df

Using run: Run('TransactionLoadFlow/1713994043337276')


### Investigate Largest Merchants
Given the task at hand is to focus on correct classification of the largest merchants, I look into various data cleaning options to improve the quality of the large merchant descriptions.

In [3]:
raw_df["merchantName"].value_counts()

merchantName
7-ELEVEN                     58684
APPLE.COM/BILL               57818
UBER   TRIP                  35204
APPLE CASH SENT MONEY        31584
APPLE CASH BALANCE ADD       23958
                             ...  
SQ *OMS BELTON                   1
CASH APP*JAYLAN*ADD CA           1
UNCLE SAMS GOOD NATURA           1
WAL Wal-Mart Super 840226        1
CASH APP*LAURA FASNACH           1
Name: count, Length: 1625233, dtype: int64

In [4]:
raw_df["description"].apply(lambda _:_.lower().split()[0]).value_counts()

description
cash                    477846
wal-mart                343655
wm                      339154
mcdonald's              240863
uber                    183684
                         ...  
spontaneity                  1
amazon.com*kq68g3993         1
amazon.com*tx7f34m82         1
doordash*fishbones           1
amazon.com*b06zf4ua3         1
Name: count, Length: 225500, dtype: int64

### Description Data Cleaning
I choose to focus on the description field, because it is the rawest form. I can't view the logic used to get from description to merchantName, so I don't have a reason to trust it, and I therefore focus on the raw description data. I might use merchantName as a proxy label to compare against later in the work. Based upon an initial view into the description data, I will carry out the following data cleaning steps.
* Lowercase the description data
* Replace amazon variations (amzn) with amazon
* Replace walmart variations (wal mart/wm supercenter) with wal-mart
* Clean up the dollar store variations (dollar tree/dollar general/family dollar)
* Expand text for doordash/square/paypal to make them more expressive